In [2]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_json('//content//script-bag-of-words.json')

In [6]:
df.tail()

,episodeAlt,seasonNum,episodeNum,episodeTitle,text
68,S8E2,8,2,A Knight of the Seven Kingdoms,"[{'name': 'Daenerys Targaryen', 'text': 'About..."
69,S8E3,8,3,The Long Night,"[{'name': 'Northman #1', 'text': 'Oi!'}, {'nam..."
70,S8E4,8,4,The Last of the Starks,"[{'name': 'Jon Snow', 'text': 'And Everyone It..."
71,S8E5,8,5,The Bells,"[{'name': 'Lord Varys', 'text': 'And? Come Not..."
72,S8E6,8,6,The Iron Throne,"[{'name': 'Tyrion Lannister', 'text': 'I'll fi..."


In [14]:
dialouge = {}
for index, row in df.iterrows():
  for item in row['text']:
    if item['name'] in dialouge:
      #append
      dialouge[item['name']] = dialouge[item['name']] + item['text']
    else:
      #create character
      dialouge[item['name']] = item['text'] + " "

In [15]:
len(dialouge)

817

In [16]:
new_df = pd.DataFrame()
new_df['character'] = dialouge.keys()
new_df['words'] = dialouge.values()

In [19]:
new_df.iloc[:,0:3].head()

,character,words
0,Will,"Easy, boy. I've I've Wildlings a a do ever in ..."
1,Waymar Royce,One They're What a and another before d'you ea...
2,Gared,Wall. We back head should the to Our They We W...
3,Jon Snow,Father's Go on. watching. And mother. yourBran...
4,Septa Mordane,"Fine Well always. as done. work, I Quite beaut..."


In [20]:
new_df['num_words'] = new_df['words'].apply(lambda x:len(x.split()))

In [21]:
new_df = new_df.sort_values('num_words',ascending=False)

In [22]:
new_df = new_df.head(100)

In [23]:
new_df.shape

(100, 3)

In [24]:
new_df.head()

,character,words,num_words
17,Tyrion Lannister,It Mmh. Northern about girls. is say the they ...,25924
13,Cersei Lannister,And And Casterly One Rock. When about afraid. ...,14294
3,Jon Snow,Father's Go on. watching. And mother. yourBran...,11488
20,Daenerys Targaryen,We've a and anything. asked been for for guest...,11202
12,Jaime Lannister,"As I It's brother, duty feel it's much. my sho...",10823


In [25]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words='english')

In [26]:
embeddings = cv.fit_transform(new_df['words']).toarray()

In [27]:
from sklearn.manifold import TSNE

In [28]:
tsne = TSNE(n_components=2, verbose=1, random_state=123)
z = tsne.fit_transform(embeddings)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 100 samples in 0.002s...
[t-SNE] Computed neighbors for 100 samples in 0.417s...
[t-SNE] Computed conditional probabilities for sample 100 / 100
[t-SNE] Mean sigma: 14.377887
[t-SNE] KL divergence after 250 iterations with early exaggeration: 60.280827
[t-SNE] KL divergence after 850 iterations: 0.294612


In [29]:
z.shape

(100, 2)

In [30]:
new_df['x'] = z.T[0]
new_df['y'] = z.T[1]

In [31]:
new_df

,character,words,num_words,x,y
17,Tyrion Lannister,It Mmh. Northern about girls. is say the they ...,25924,-2.584323,-0.540657
13,Cersei Lannister,And And Casterly One Rock. When about afraid. ...,14294,-2.324203,-0.528935
3,Jon Snow,Father's Go on. watching. And mother. yourBran...,11488,-2.244610,-0.625231
20,Daenerys Targaryen,We've a and anything. asked been for for guest...,11202,-1.827532,-0.935853
12,Jaime Lannister,"As I It's brother, duty feel it's much. my sho...",10823,-2.141694,-0.532025
...,...,...,...,...,...
132,Rickard Karstark,Castle I'll King North! Red The They and can c...,466,5.382496,-0.433401
57,Syrio Forel,"Tomorrow You are at be boy. here late, midday....",462,4.585623,-1.307379
112,Kevan Lannister,Tyrion. Catelyn Golden Jaime Lords River River...,460,4.526544,-0.932279
365,Mace Tyrell,"From Grace, House Margaery May Reach, Tyrell Y...",450,4.339713,-1.036100


In [32]:
import plotly.express as px
fig = px.scatter(new_df.head(25), x="x", y="y", color="character")
fig.show()